### Integer Linear Program for Graph Coloring

In [1]:
import numpy as np
import gurobipy as gp
from tqdm import tqdm
import pandas as pd
import time 

In [2]:
graphs = [25] + [i for i in range(50, 1000, 50)] + [i for i in range(1000, 10001, 500)]
c = 4 # max number of used colors 

In [3]:
def read_data_edge_list(filename): 
    ''' 
    reads data from edge list input file and returns adjacency matrix 
    '''

    file = open(filename, 'r')
    content = file.readlines()

    n = int(content[0]) # number of vertices 
    m = int(content[1]) # number of edges 

    adj_mat = np.zeros((n,n)) # adjacency matrix 

    # range(2, 2*m+2) for .pg files 
    # range(2, m+2) for .txt files
    for l in range(2, m+2): 
        line = content[l]
        line_split = line.split(' ')
        u = int(line_split[0])
        v = int(line_split[1])
        adj_mat[u,v] = 1

    return adj_mat

In [4]:
def create_model(edges: np.array, n: int, c: int): 
    ''' 
    creates an ILP from adjacency matrix 
    '''

    # create new model
    model = gp.Model("GraphColoring")
    model.setParam('OutputFlag', 0)

    # add the decision variables 
    x = {}
    w = {}

    for j in range(c): 
        for i in range(n): 
            # x_ij = 1 <=> node i is assigned color j
            x[i, j] = model.addVar(vtype=gp.GRB.BINARY, name=f"x[{i},{j}]")
        # w_j = 1 <=> at least one node is assigend color j 
        w[j] = model.addVar(vtype=gp.GRB.BINARY, name="w[%d]"%j)

    model.update()

    # set objective function: minimize sum of w_i
    w_vars = [v for v in model.getVars() if 'w' in v.varName]
    model.setObjective(gp.LinExpr(np.ones(c), w_vars), gp.GRB.MINIMIZE)

    # ensure that every node is assigned exactly one color
    model.addConstrs((np.sum([x[i, j] for j in range(c)]) == 1 for i in range(n)), name=f"constr_cx[{i},{j}]") 

    # ensure that adjacent nodes have different colors 
    for k in range(n): 
        for l in range(k, n): 
            if edges[k, l]: 
                model.addConstrs((x[k, j] + x[l, j] <= 1 for j in range(c)), name=f"constr_x[{k}]x[{l}]")

    # set w_j 
    model.addConstrs((x[i, j] <= w[j] for j in range(c) for i in range(n)), name=f"constr_xw[{i},{j}]") 

    model.update()

    return model 

In [5]:
def write_to_csv(filename: str, data: float, n: int): 
    ''' 
    writes the results to a csv file 
    '''

    df = pd.read_csv(filename)
    df.set_index('Graph', inplace=True)
    index = 'graph_'+str(n)
    df.loc[index, 'ILP'] = np.round(data, 5)
    df.to_csv(filename, index=True)

    return 

In [ ]:
for n in graphs: 

    edges = read_data_edge_list('./generated/graph_'+str(n)+'.txt')
    c_min = 10

    start = time.time()

    model = create_model(edges, n, c)

    for i in range(5): 
        # optimize the model
        model.optimize()
        c_num = model.objVal
        c_min = np.min([c_min, c_num])

    end = time.time()

    print("Colors for graph_"+str(n)+"=", c_min)

    # determine runtime 
    runtime = (end-start)/5 

    write_to_csv('./results/results_runtime.csv', runtime, n)
    write_to_csv('./results/results_coloring.csv', c_min, n)